## Preparação do ambiente

- Definição da sessão spark
- Conexão com o Drive

In [281]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [282]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('Challenge DataScience - 2 Ed')\
                    .getOrCreate()
spark

In [283]:
from google.colab import drive 
drive.mount('my-drive')

Drive already mounted at my-drive; to attempt to forcibly remount, call drive.mount("my-drive", force_remount=True).


## Manipulações iniciais da base já tratada
  - A partir do dataset já inicialmente tratado a partir do conjunto de dados bruto fornecido no início do projeto, o objetivo será efetuar algumas manipulações básicas (essenciais para os métodos de Machine Learning) e iniciar a construção do modelo de regressão

In [284]:
# começo retomando a base de dados tratados construída anteriormente;
# comparo a base tratada fornecida pelos instrutores da alura com a base construída por mim
# para validação de resultados

df = spark.read.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/parquet'
)


In [285]:
df.show(5)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+---------+----------+----------+----+-----+-----+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|     caracteristicas|   bairro|      zona|condominio|iptu| tipo|valor|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+---------+----------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a...|       Usado| Apartamento|Residencial|        43|       43|      2|  null|        1|   1|    3|[Academia, Churra...|Paciência|Zona Oeste|       245|null|Venda|15000|
|085bab2c-87ad-452...|       Usado| Apartamento|Residencial|        42|       42|      2|  null|        1|   1|    2|[Churrasqueira, P...|Paciência|Zona Oeste|         0|   0|Venda|15000|
|18d22cbe-1b86-476...|       Usado| Apartamento|Residencial|

In [286]:
df.count()

66348

In [287]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: long (nullable = true)
 |-- area_util: long (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: long (nullable = true)
 |-- iptu: long (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: long (nullable = true)



- De acordo com o Schema observado, as primeiras manipulações que estaremos interessados em fazer ocorrem com respeito aos datatypes de cada coluna.
  - As colunas "andar", "banheiros", "suites" e "quartos" estão registradas como LongInt. A princípio, para esses casos, apenas o IntegerType já é suficiente.

  - As colunas "area_util", "condominio", "iptu" e "valor", por outro lado, devem ficar registradas com o tipo DoubleType.

In [288]:
from pyspark.sql import functions as f 
from pyspark.sql.types import DoubleType, IntegerType

columns_to_set_DoubleType = ['area_util', 'condominio', 'iptu', 'valor']
columns_to_set_IntegerType = ['andar', 'banheiros', 'suites', 'quartos']

for c in columns_to_set_DoubleType:
  df = df.withColumn(c, f.col(c).cast(DoubleType()))

for c in columns_to_set_IntegerType:
  df = df.withColumn(c, f.col(c).cast(IntegerType()))

df.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- area_total: long (nullable = true)
 |-- area_util: double (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- vaga: long (nullable = true)
 |-- andar: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



- Nos foi passada a informação de que alguns registros da coluna "caractersticas" são listas vazias. Devemos, portanto, eliminar esses registros do nosso conjunto de dados
  - A ideia será usar o método ".getItem(0)" para verificar quais registros têm esse primeiro valor nulo. Se ele for nulo, que dizer que a lista da linha correspondente é vazia


In [289]:
# contando quantos elementos contém listas vazias na coluna de características
df.select('*').where(f.isnull( f.col('caracteristicas').getItem(0) )).count()

12665

- Como são muitos dados (~10 000, mesma ordem de grandeza da quantidade total de registros) não podemos simplesmente descartá-los do nosso dataset; as demais colunas associadas a essas linhas podem conter informações relevantes para a construção posterior dos modelos de regressão.
- Para contornar esse problema, portanto, ajusto todas essas colunas com valor nulo (null)

In [290]:
# outra maneira de detectar esses caras utilizando a função "size" do pyspark.sql
df.select('*').filter( f.size( f.col('caracteristicas') ) == 0 ).count()

12665

In [291]:
# verificação dos arrays vazios (antes do tratamento)
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                  []|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



In [292]:
# Faço a atualização desses dados utilizando o case when/else (f.when().otherwise, no caso do pyspark)

# detalhes da logica abaixo: quando o "len" da coluna caracteristicas for 0 (i.e., quando o array do campo correspondente for nulo),
# adicionamos o valor null (None para o dataframe spark no python); caso contrário, ajustamos o registro da própria coluna
df = df.withColumn('caracteristicas', f.when( f.size( f.col('caracteristicas') ) == 0, None ).otherwise( f.col('caracteristicas') ) )

# Poderíamos também estruturar esse mesmo update utilizando o método "getIem(0)" (verificando quando valor logo do primeiro index fosse nulo, o que 
# indicaria uma lista vazia)
# (utilizo uma sintaxe diferente no no argumento de aplicação das funções de tratamento apenas por prática)
#df = df.withColumn('caracteristicas', f.when( df['caracteristicas'].getItem(0).isNull(), None ).otherwise( df['caracteristicas'] ) )

In [293]:
#teste = df.select('*')
#teste = teste.withColumn('caracteristicas', f.when( f.col('caracteristicas').getItem(0).isNull(), 0 ).otherwise( f.col('caracteristicas') ) )

# o método "case when / else " (when / otherwise, no pyspark) não funcionou pois estamos colocando dois tipos de dados diferentes como saída:
# inteiro se a condição verificar verdadeira e array se a condição verificar falsa;
# a lógica está correta mas o spark não consegue efetuar esse tratamento

In [294]:
# validação da função de tratamento aplicada
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                null|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



- Verifico a presença de registros nulos ao longo das colunas do nosso dataset e efetuo os tratamentos correspondentes

In [295]:
array_columns_to_check = df.columns

array_columns_to_check.remove('caracteristicas') 
# excluo a coluna de caracteristicas desse processo automatizado de contagem, mas sabemos já sabemos que ela contém um total de 
# 12 665 dados nulos (ajustes efetuados na célula anterior)

df.select([ f.count( f.when( f.isnull(col) | f.isnan(col) , 1 ) ).alias(col) for col in array_columns_to_check ] ).show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|      9186|        0|      0|  5544|        0|3008|    0|     0|   0|      2347|7155|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [296]:
# verifico se a contagem anterior se refere somente aos dados null, ou se dados tipo nan contribuiram para o resultado final
df.select( [f.count( f.when( f.isnull(c), 1 ) ).alias(c) for c in array_columns_to_check] ) .show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|      9186|        0|      0|  5544|        0|3008|    0|     0|   0|      2347|7155|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [297]:
# preenchimento dos dados nulos com valores 0
df = df.na.fill(0)

In [298]:
# efetuo novamente a contagem por fins de validação
df.select( [f.count( f.when( f.isnull(c), 1 ) ).alias(c) for c in array_columns_to_check] ) .show()

+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
| id|tipo_anuncio|tipo_unidade|tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|bairro|zona|condominio|iptu|tipo|valor|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+
|  0|           0|           0|       0|         0|        0|      0|     0|        0|   0|    0|     0|   0|         0|   0|   0|    0|
+---+------------+------------+--------+----------+---------+-------+------+---------+----+-----+------+----+----------+----+----+-----+



In [299]:
# verifico se os registros da coluna de caracteristicas foram corrigidos
df.select('caracteristicas').distinct().orderBy('caracteristicas', ascending=True).show(5)

+--------------------+
|     caracteristicas|
+--------------------+
|                null|
|          [Academia]|
|[Academia, Animai...|
|[Academia, Animai...|
|[Academia, Animai...|
+--------------------+
only showing top 5 rows



- Ao que podemos ver os registros nulos da coluna de caracteristicas não estão sendo corrigidos pelas nossas ferramentas usuais. Por enquanto isso não é um problema, pois a nossa ideia será quebrar os registros desses arrays em dummy variables no momento em que estivermos preparando o dataset para construção do modelo de regressão.

In [300]:
df.select('zona').distinct().collect()

[Row(zona='Zona Norte'),
 Row(zona='Zona Oeste'),
 Row(zona='Zona Central'),
 Row(zona='Zona Sul'),
 Row(zona='')]

In [301]:
# começo criando as dummy variables para a coluna de zonas
df_zona = df\
            .groupBy('id')\
            .pivot('zona')\
            .agg(f.lit(1))\
            .na\
            .fill(0)

df_zona.show(5)


+--------------------+---+------------+----------+----------+--------+
|                  id|   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+---+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|  0|           0|         0|         0|       1|
|02fba6ef-a691-442...|  0|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|  0|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|  0|           0|         0|         0|       1|
|82707939-71bd-40c...|  0|           0|         0|         0|       1|
+--------------------+---+------------+----------+----------+--------+
only showing top 5 rows



In [302]:
df_zona = df_zona.drop('')
# o método ".drop" está sendo incluído para excluir uma coluna vazia que está sendo criada, correspondente a uma zona registrada como " '' "

df_zona.show(5)

+--------------------+------------+----------+----------+--------+
|                  id|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|           0|         0|         0|       1|
|02fba6ef-a691-442...|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|           0|         0|         0|       1|
|82707939-71bd-40c...|           0|         0|         0|       1|
+--------------------+------------+----------+----------+--------+
only showing top 5 rows



- Agora que o dataframe auxiliar de dummy variables correspondente à coluna "zona" está feito, desenvolvo um tratamento da coluna "características" (formato array)

In [303]:
# defino um dataframe auxiliar incluindo somente as colunas que vamos trabalhar, com a finalidade de não sujar o nosso dataframe original
df_aux = df\
            .select('id', 'caracteristicas')

# incluo uma nova coluna nesse dataframe utilizando o método ".explode()", que basicamente funciona extraindo a os arrays da coluna de características
# e colocando-os numa mesma coluna para cada id correspondente.

# em outras palavras: se uma coluna com id "x" possuía inicialmente 5 valores dentro do array da coluna 'caracteristicas', então
# serão criadas 5 novas linhas com o id "x", uma para cada valor dentro do array da coluna 'caracteristicas'.

df_aux = df_aux.withColumn('caracteristicas_values', f.explode( f.col('caracteristicas') ))
df_aux.show(n = 10, truncate=False)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+----------------------+
|id                                  |caracteristicas                                                                                                                |caracteristicas_values|
+------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+----------------------+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, Churrasqueira, Playground, Salão de festas, Condomínio fechado, Portão eletrônico, Portaria 24h, Animais permitidos]|Academia              |
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, Churrasqueira, Playground, Salão de festas, Condomínio fechado, Portão eletrônico, Portaria 24h, Animais permitidos]|Churrasqueira         |
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|[Academia, C

In [304]:
# agora efetuo um pivot da nova coluna criada e defino um novo dataframe auxiliar
df_caracteristicas = df_aux\
                          .groupBy('id')\
                          .pivot('caracteristicas_values')\
                          .agg(f.lit(1))\
                          .na\
                          .fill(0)

df_caracteristicas.show(10)

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

- Uma vez feitos esses tratamentos, junto tudo no nosso dataset inicial

In [305]:
df.count()

66348

In [306]:
df = df.join(other= df_zona , on= 'id', how= 'inner')
df = df.join(other= df_caracteristicas, on= 'id', how= 'inner')

df.show(10)

+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+----------+----------+------+-----+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|tipo_anuncio|tipo_unidade|   tipo_uso|area_total|area_util|quartos|suites|banheiros|vaga|andar|     caracteristicas|              bairro|      zona|condominio|  iptu| tipo|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+------------+------------+-----------+----------+---------+-------+------+---------+----+-----+--------------------+--------------------+----------+----------+------+-----+---------+------------+----------+--

In [307]:
df.count()

53683

- Podemos ver que estamos perdendo cerca de ~ 10 000 dados após os joins de cruzamento com os dataframes auxiliares criados. Isso está ocorrendo devido aos registros nulos que temos no nosso dataframe de caracteristicas. Como não há características para "pivotar", essas linhas são simplesmente apagadas do dataframe.

In [308]:
# uma maneira de testar essa hipótese, é contar a quantidade de registros antes e após a aplicação do argumento "explode" e pivot
teste = spark.read.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset-tratado/parquet'
)
teste = teste.withColumn('caracteristicas', f.when( f.size( f.col('caracteristicas') ) == 0, None ).otherwise( f.col('caracteristicas') ) )


qtd_total_inicial = teste.count()
qtd_nulos_inicial = teste.filter( f.col('caracteristicas').isNull() ).count()

# agora aplico os métodos explode/pivot na coluna da características
# (a expectativa é que o explode ainda mantenha os registros nulos, mas que eles sumam durante a aplicação do pivot)
teste = teste.withColumn('caracteristicas_values', f.explode( f.col('caracteristicas') ))

qtd_nulos_logo_apos_explode = teste.filter( f.col('caracteristicas').isNull() ).count()

teste_aux = teste.groupBy('id').pivot('caracteristicas_values').agg(f.lit(1)).na.fill(0)
qtd_dados_tabela_pivot = teste_aux.count()

teste = teste.join(other= teste_aux, on= 'id', how= 'inner')
qtd_total_final = teste.count()
qtd_nulos_final = teste.filter( f.col('caracteristicas').isNull() ).count()

print('Quantidade inicial total de dados no dataset: {}'.format(qtd_total_inicial))
print('Quantidade inicial de nulos ("caracteristicas") no dataset: {}'.format(qtd_nulos_inicial))
print('Quantidade de nulos ("caracteristicas") após o método "explode": {}'.format(qtd_nulos_logo_apos_explode))
print('Quantidade total de dados na tabela pivot: {}'.format(qtd_dados_tabela_pivot))
print('Quantidade total de dados após o join: {}'.format(qtd_total_final))
print('Quantidade total de nulos ("caracteristicas") após o join: {}'.format(qtd_nulos_final))

Quantidade inicial total de dados no dataset: 66348
Quantidade inicial de nulos ("caracteristicas") no dataset: 12665
Quantidade de nulos ("caracteristicas") após o método "explode": 0
Quantidade total de dados na tabela pivot: 53683
Quantidade total de dados após o join: 325879
Quantidade total de nulos ("caracteristicas") após o join: 0


- Depois de efetuados esses tratamentos, começo o processo de seleção / definição das features que serão aplicadas no modelo. 
  - As colunas "tipo_anuncio", "tipo_unidade", "tipo_uso" e "tipo" possuem apenas um único registro, pois já efetuamos filtros dos valores dessas colunas nos tratamentos iniciais da base de dados da semana 1. Elas podem ser descartadas (serão irrelevantes para o modelo)
  - As colunas "area_total" e "area_util" são dois casos peculiares. A princípio o ideal seria selecionarmos todas as features de interesse e montar um mapa de calor de correlação. Mas essas duas colunas são claramente correlacionadas. Não faz sentido continuarmos carregando as duas. Apesar da forte correlação entre ambas e muitos dos registros incluídos na base serem iguais nessas duas colunas, muito provavelmente a "area_total" tem um impacto maior na definição do preço do imóvel. Isso precisaria ser discutido com as demais equipes do projeto (não caberia à equipe de Data Science e Analytics tomar essa decisão), mas pensemos por um momento: mesmo que a área útil de uma residência seja pequena, se o terreno for grande a expectativa é que, de fato, o valor do imóvel aumente.
    - A conclusão, portanto, seria deixaremos de continuar carregando a coluna "area_util" e efetuaremos um filtro para excluir aqueles registros de "area_total" com valor 0. Entretanto essa coluna possui muitos valores 0, tal como pode ser observado a seguir. De fato, se retrocedermos o nosso código, veremos que ela foi uma das colunas atingidas pelo nosso método ".na.fill(0)" para preenchimento de valores nulos. Por uma questão de ganho de informação e maior confiabilidade dos resultados, excluiremos a coluna "area_total" e seguiremos trabalhando somente com a coluna "area_util".

- As informações da coluna "características" já foram ajustadas, então não há necessidade de continuar com essa coluna também, bem como as colunas "zona" e "bairro" (cujas informações já estão armazenadas nos campos de zona)

- A coluna "id" também não será de grande relevância para a construção do nosso modelo

In [309]:
df.select('area_total', 'area_util', 'valor').limit(30).show()

+----------+---------+--------+
|area_total|area_util|   valor|
+----------+---------+--------+
|        43|     43.0| 22999.0|
|        47|     47.0|138000.0|
|     17089|     45.0|140000.0|
|        48|     48.0|150000.0|
|         0|     55.0|160000.0|
|        70|     70.0|159000.0|
|        50|     48.0|170000.0|
|        40|     40.0|189000.0|
|        50|     50.0|185000.0|
|        45|     45.0|199000.0|
|        48|     48.0|205933.0|
|        44|     44.0|206000.0|
|        55|     55.0|219130.0|
|        60|     60.0|230000.0|
|        51|     51.0|239000.0|
|        50|     50.0|260000.0|
|        96|     96.0|260000.0|
|        48|     48.0|251800.0|
|        55|     55.0|260000.0|
|        48|     48.0|270000.0|
+----------+---------+--------+
only showing top 20 rows



In [310]:
# então, aplico os drops das colunas irrelevantes para os nossos modelos
df = df.drop('tipo_unidade', 'tipo_anuncio', 'tipo_uso', 'area_total', 'tipo', 'bairro', 'zona', 'caracteristicas', 'id')

In [311]:
df.show(5)

+---------+-------+------+---------+----+-----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|area_util|quartos|suites|banheiros|vaga|andar|condominio|  iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+---------+-------+------+---------+----+-----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|    410.0|      4|     1|        3|   1|    0|    3300.0|5780.0|4600000.0|           0|         0|         0|       1|       0|                 0|            0|                 0|       1|      0|         0|           0|           

In [312]:
# Começo a definição das nossas features e a transformação do dataframe
from pyspark.ml.feature import VectorAssembler

X = [col for col in df.columns] # variáveis /features de interesse (colunas irrelevantes já removidas)
vec_ass = VectorAssembler(inputCols= X, outputCol= 'features') # classe de assembling do vetor de features (transformadora; não precisa de fit)

df = vec_ass.transform(df) # dataframe completo tratado
df = df.withColumnRenamed('valor', 'label')

df_ML = df.select('features', 'label') # dataframe contendo somente a coluna de features / labels (além do identificador id)

In [314]:
df.show(10)

+---------+-------+------+---------+----+-----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+
|area_util|quartos|suites|banheiros|vaga|andar|condominio|  iptu|    label|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|
+---------+-------+------+---------+----+-----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+
|    410.0|      4|     1|        3|   1|    0|    3300.0|5780.0|4600000.0|           0|         0|         0|       1|       0|                 0|            0|        

In [315]:
df_ML.show(10)

+--------------------+---------+
|            features|    label|
+--------------------+---------+
|(23,[0,1,2,3,4,6,...|4600000.0|
|(23,[0,1,3,4,5,6,...| 360000.0|
|(23,[0,1,2,3,4,8,...|1200000.0|
|(23,[0,1,3,6,7,8,...| 750000.0|
|(23,[0,1,3,4,6,7,...|1025000.0|
|[115.0,3.0,0.0,2....|1100000.0|
|[65.0,2.0,0.0,1.0...| 593036.0|
|[74.0,2.0,1.0,2.0...|1031576.0|
|(23,[0,1,2,3,5,6,...| 800000.0|
|(23,[0,1,2,3,4,6,...| 497000.0|
+--------------------+---------+
only showing top 10 rows



In [313]:
# depois de todos os tratamentos efetuados anteriormente são essenciais para aplicação em treino/teste nos modelos de ML.
# para dinamizar e facilitar os processos futuros, salvo essa base e deixo guardada para quando mais for necessário

df.write.parquet(
    '/content/my-drive/MyDrive/Formações Alura/DataScience/Challenge-2ed/dados-e-arquivos-tratados/dataset_tratado_para_ML_parquet',
    mode = 'overwrite'
)

## Construindo e estudando os modelos de Machine Learning
- Nessa seção estudaremos os 3 principais tipos de modelos de regressão para bases de dados tratadas (Regressão Linear, Árvore de Decisão e Random Forest) com a finalidade de verificar qual deles melhor se adequa à nossa base de dados

### 1) Regressão Linear

### 2) Árvore de Decisão

### 3) Random Forest

### 4) Definição do melhor modelo e implementação de otimização